# MRI images Brain Tumor Classification


In [ ]:
import keras
keras.__version__
from keras import backend as K
#K.tensorflow_backend._get_available_gpus()
import tensorflow as tf

from tensorflow.python.client import device_lib
local_device_protos = device_lib.list_local_devices()

for val in local_device_protos:
  print(val.device_type)


## Connect to TPU


In [ ]:
import keras
keras.backend.set_session("TPU")

In [ ]:
pip install scikit-learn

In [ ]:
pip install seaborn

In [ ]:
pip install plotly

In [ ]:
pip install missingno

In [ ]:

import sys
import os
import math

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from matplotlib import rcParams
rcParams['figure.dpi'] = 300
%matplotlib inline
import seaborn as sns
import missingno as msno
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import *

from PIL import Image, ImageEnhance
from tensorflow.keras.preprocessing.image import *

In [ ]:
import pathlib
import tensorflow as tf

train_dir = pathlib.Path('/kaggle/input/brain-tumor-mri-dataset/Training')
test_dir = pathlib.Path('/kaggle/input/brain-tumor-mri-dataset/Testing')
img_height=224
img_width=224

In [ ]:
train_ds  = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=128)


In [ ]:
test_ds  = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=128)


In [ ]:
val_ds  = tf.keras.utils.image_dataset_from_directory(
  test_dir,
  validation_split=None,
  subset=None,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=128)


## Visualize the data distribution

In [ ]:
ROOT_DIR = r"/kaggle/input/brain-tumor-mri-dataset"
TRAIN_DIR = os.path.join(ROOT_DIR, 'Training')
TEST_DIR = os.path.join(ROOT_DIR, 'Testing')
assert os.path.isdir(ROOT_DIR) and os.path.isdir(TRAIN_DIR) and os.path.isdir(TEST_DIR)
TUMOR_CLASS = ['meningioma', 'glioma', 'pituitary', 'notumor']
IMAGE_DATA_PATHS = [os.path.join(TRAIN_DIR, tumor_class) for tumor_class in TUMOR_CLASS]
TEST_DATA_PATHS = [os.path.join(TEST_DIR, tumor_class) for tumor_class in TUMOR_CLASS]

In [ ]:
TEST_DATA_PATHS

In [ ]:
data_distribution_count = pd.Series([len(os.listdir(path)) for path in TEST_DATA_PATHS if os.path.exists(path) and os.path.isdir(path)],index = TUMOR_CLASS)
data_distribution_count

In [ ]:
data_distribution_count = pd.Series([len(os.listdir(path)) for path in IMAGE_DATA_PATHS if os.path.exists(path) and os.path.isdir(path)],index = TUMOR_CLASS)
data_distribution_count

In [ ]:
fig, axis = plt.subplots(figsize = (13, 5))
axis.grid(True, alpha = 0.1)
axis.set_title("Data Distribution Percentage (%)", fontsize = 14)
sns.barplot(x = ['\n'.join(curr_index.strip().split('_')).title() for curr_index in data_distribution_count.index],
            y = 100 * (data_distribution_count / data_distribution_count.sum()), ax = axis)
axis.set_xlabel("Tumor Class", fontsize = 12)
axis.set_ylabel("% Total Observations", fontsize = 12)
axis.tick_params(which = 'major', labelsize = 12)
axis.text(2.5, 37, f'Total Observations: {data_distribution_count.sum()}', fontdict = dict(size = 12))
sns.despine()

In [ ]:
BRIGHTNESS_FACTOR = 1.7
fig, axes = plt.subplots(nrows = 1, ncols = 4, figsize = (18, 5))
axes = axes.flatten()
fig.suptitle("Brain Tumor Images", fontsize = 16, fontdict = dict(weight = 'bold'), y = 1.04)
for curr_title, filename, curr_axis in zip(TUMOR_CLASS, IMAGE_DATA_PATHS, axes[:4]):
    curr_image = Image.open(os.path.join(filename, os.listdir(filename)[2]))
    img_enhancer = ImageEnhance.Brightness(curr_image)
    curr_axis.imshow(img_enhancer.enhance(BRIGHTNESS_FACTOR))
    curr_axis.set_title(" ".join(curr_title.split('_')).title(), fontsize = 14)

fig.tight_layout()
sns.despine()

## Image Augmentation

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

In [ ]:
import numpy as np
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

### 1. Inception V3

In [ ]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from keras.layers import GlobalAveragePooling2D, Flatten, BatchNormalization, Dense, Dropout
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D

# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)


In [ ]:

# instantiating the model in the strategy scope creates the model on the TPU
with tpu_strategy.scope():
    base_model1 = InceptionV3(
                    input_shape=(224, 224, 3),
                    weights='imagenet',
                    include_top=False) # define your model normally
    for layer in base_model1.layers[:10]:
        layer.trainable = False
    x = base_model1.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.4)(x)
    predictions1 = Dense(4, activation='softmax')(x)
    model1 = Model(inputs=base_model1.inputs, outputs=predictions1)
    model1.compile(optimizer='adam',loss=tf.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])



In [ ]:
history1=model1.fit(
  train_ds,
  validation_data=test_ds,
  epochs=25)

model1.save('model1.h5')

In [ ]:
inceptionv3_evaluation = model1.evaluate(val_ds)
inceptionv3_evaluation[1]*100

In [ ]:
import matplotlib.pyplot as plt
acc = history1.history['accuracy']
val_acc = history1.history['val_accuracy']
loss = history1.history['loss']
val_loss = history1.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

### 2. VGG16

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D

with tpu_strategy.scope():
    base_model2 = VGG16(
                        input_shape=(224, 224, 3),
                        weights='imagenet',
                        include_top=False)
    # Freeze the first 10 layers
    for layer in base_model2.layers:
        layer.trainable = False
    x = base_model2.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.4)(x)
    predictions2 = Dense(4, activation='softmax')(x)
    model2 = Model(inputs=base_model2.inputs, outputs=predictions2)
    model2.compile(optimizer='adam',loss=tf.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [ ]:
history2=model2.fit(
  train_ds,
  validation_data=test_ds,
  epochs=25)

model2.save('model2.h5')

In [ ]:
vgg16_evaluation = model2.evaluate(val_ds)
vgg16_evaluation[1]*100

In [ ]:
import matplotlib.pyplot as plt
acc = history2.history['accuracy']
val_acc = history2.history['val_accuracy']
loss = history2.history['loss']
val_loss = history2.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

### 3. VGG19

In [ ]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D

with tpu_strategy.scope():

    base_model3 = VGG19(
                        input_shape=(224, 224, 3),
                        weights='imagenet',
                        include_top=False)
    # Freeze the first 10 layers
    for layer in base_model3.layers[:10]:
        layer.trainable = False
    x = base_model3.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    predictions3 = Dense(4, activation='softmax')(x)
    model3 = Model(inputs=base_model3.inputs, outputs=predictions3)
    model3.compile(optimizer='adam',loss=tf.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [ ]:
history3=model3.fit(
  train_ds,
  validation_data=test_ds,
  epochs=25)

model3.save('model3.h5')

In [ ]:
vgg19_evaluation = model3.evaluate(val_ds)
vgg19_evaluation[1]*100

In [ ]:
import matplotlib.pyplot as plt
acc = history3.history['accuracy']
val_acc = history3.history['val_accuracy']
loss = history3.history['loss']
val_loss = history3.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

### 4. RESNET50

In [ ]:
from tensorflow.keras.applications import ResNet50

with tpu_strategy.scope():

    base_model4 = ResNet50(
                        input_shape=(224, 224, 3),
                        weights='imagenet',
                        include_top=False)
    # Freeze the first 10 layers
    for layer in base_model4.layers[:10]:
        layer.trainable = False
    x = base_model4.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.4)(x)
    predictions4 = Dense(4, activation='softmax')(x)
    model4 = Model(inputs=base_model4.inputs, outputs=predictions4)
    model4.compile(optimizer='adam',loss=tf.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [ ]:
history4=model4.fit(
  train_ds,
  validation_data=test_ds,
  epochs=25)

model4.save('model4.h5')

In [ ]:
resnet50_evaluation = model4.evaluate(val_ds)
resnet50_evaluation[1]*100

In [ ]:
import matplotlib.pyplot as plt
acc = history4.history['accuracy']
val_acc = history4.history['val_accuracy']
loss = history4.history['loss']
val_loss = history4.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

### 5. Inception-ResNet-v2

In [ ]:
with tpu_strategy.scope():

    base_model5 = tf.keras.applications.InceptionResNetV2(
                        input_shape=(224, 224, 3),
                        weights='imagenet',
                        include_top=False)
    # Freeze the first 10 layers
    for layer in base_model5.layers[:10]:
        layer.trainable = False
    x = base_model5.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.4)(x)
    predictions5 = Dense(4, activation='softmax')(x)
    model5 = Model(inputs=base_model5.inputs, outputs=predictions5)
    model5.compile(optimizer='adam',loss=tf.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [ ]:
history5=model5.fit(
  train_ds,
  validation_data=test_ds,
  epochs=25)

model5.save('model5.h5')

In [ ]:
inceptionresnetv2_evaluation = model5.evaluate(val_ds)
inceptionresnetv2_evaluation[1]*100

In [ ]:
import matplotlib.pyplot as plt
acc = history5.history['accuracy']
val_acc = history5.history['val_accuracy']
loss = history5.history['loss']
val_loss = history5.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

### 6. DenseNet-201

In [ ]:
with tpu_strategy.scope():

    base_model6 = tf.keras.applications.DenseNet201(
                        input_shape=(224, 224, 3),
                        weights='imagenet',
                        include_top=False)
    # Freeze the first 10 layers
    for layer in base_model6.layers[:10]:
        layer.trainable = False
    x = base_model6.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.4)(x)
    predictions6 = Dense(4, activation='softmax')(x)
    model6 = Model(inputs=base_model6.inputs, outputs=predictions6)
    model6.compile(optimizer='adam',loss=tf.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [ ]:
history6=model6.fit(
  train_ds,
  validation_data=test_ds,
  epochs=25)

model6.save('model6.h5')

In [ ]:
densenet201_evaluation = model6.evaluate(val_ds)
densenet201_evaluation[1]*100

In [ ]:
import matplotlib.pyplot as plt
acc = history6.history['accuracy']
val_acc = history6.history['val_accuracy']
loss = history6.history['loss']
val_loss = history6.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

### 7. ResNet-101

In [ ]:
with tpu_strategy.scope():

    base_model7 = tf.keras.applications.ResNet101(
                        input_shape=(224, 224, 3),
                        weights='imagenet',
                        include_top=False)
    # Freeze the first 10 layers
    for layer in base_model7.layers[:10]:
        layer.trainable = False
    x = base_model7.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.4)(x)
    predictions7 = Dense(4, activation='softmax')(x)
    model7 = Model(inputs=base_model7.inputs, outputs=predictions7)
    model7.compile(optimizer='adam',loss=tf.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [ ]:
history7=model7.fit(
  train_ds,
  validation_data=test_ds,
  epochs=25)

model7.save('model7.h5')

In [ ]:
resnet101_evaluation = model7.evaluate(val_ds)
resnet101_evaluation[1]*100

In [ ]:
import matplotlib.pyplot as plt
acc = history7.history['accuracy']
val_acc = history7.history['val_accuracy']
loss = history7.history['loss']
val_loss = history7.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

### 8. MobileNet

In [ ]:
with tpu_strategy.scope():

    base_model8 = tf.keras.applications.MobileNet(
                        input_shape=(224, 224, 3),
                        weights='imagenet',
                        include_top=False)
    # Freeze the first 10 layers
    for layer in base_model8.layers[:10]:
        layer.trainable = False
    x = base_model8.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.4)(x)
    predictions8 = Dense(4, activation='softmax')(x)
    model8 = Model(inputs=base_model8.inputs, outputs=predictions8)
    model8.compile(optimizer='adam',loss=tf.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [ ]:
history8=model8.fit(
  train_ds,
  validation_data=test_ds,
  epochs=25)

model8.save('model8.h5')

In [ ]:
mobilenet_evaluation = model8.evaluate(val_ds)
mobilenet_evaluation[1]*100

In [ ]:
import matplotlib.pyplot as plt
acc = history8.history['accuracy']
val_acc = history8.history['val_accuracy']
loss = history8.history['loss']
val_loss = history8.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

### 9. Xception

In [ ]:
with tpu_strategy.scope():
    base_model9 = tf.keras.applications.Xception(
                        input_shape=(224, 224, 3),
                        weights='imagenet',
                        include_top=False)
    # Freeze the first 10 layers
    for layer in base_model9.layers[:10]:
        layer.trainable = False
    x = base_model9.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.4)(x)
    predictions9 = Dense(4, activation='softmax')(x)
    model9 = Model(inputs=base_model9.inputs, outputs=predictions9)
    model9.compile(optimizer='adam',loss=tf.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [ ]:
history9=model9.fit(
  train_ds,
  validation_data=test_ds,
  epochs=25)

model9.save('model9.h5')

In [ ]:
xception_evaluation = model9.evaluate(val_ds)
xception_evaluation[1]*100

In [ ]:
import matplotlib.pyplot as plt
acc = history9.history['accuracy']
val_acc = history9.history['val_accuracy']
loss = history9.history['loss']
val_loss = history9.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

### 10. ResNet-152

In [ ]:
with tpu_strategy.scope():

    base_model10 = tf.keras.applications.ResNet152(
                        input_shape=(224, 224, 3),
                        weights='imagenet',
                        include_top=False)
    # Freeze the first 10 layers
    for layer in base_model10.layers[:10]:
        layer.trainable = False
    x = base_model10.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.4)(x)
    predictions10 = Dense(4, activation='softmax')(x)
    model10 = Model(inputs=base_model10.inputs, outputs=predictions10)
    model10.compile(optimizer='adam',loss=tf.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [ ]:
history10=model10.fit(
  train_ds,
  validation_data=test_ds,
  epochs=25)

model10.save('model10.h5')

In [ ]:
resnet152_evaluation = model10.evaluate(val_ds)
resnet152_evaluation[1]*100

In [ ]:
import matplotlib.pyplot as plt
acc = history10.history['accuracy']
val_acc = history10.history['val_accuracy']
loss = history10.history['loss']
val_loss = history10.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

#### Model Checkpoint

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
# Checkpoint to save best model per epoch

model_filepath = "/kaggle/working-{epoch:02d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(
    filepath=model_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

### Building Ensemble Model

In [ ]:
# choosing only the best accuracy models

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Average
with tpu_strategy.scope():
#     model_1 = load_model('/kaggle/working/model1.h5')
#     model_1 = Model(inputs=model_1.inputs,
#                     outputs=model_1.outputs,
#                     name='name_of_model_1')

    model_2 = load_model('/kaggle/working/model2.h5')
    model_2 = Model(inputs=model_2.inputs,
                    outputs=model_2.outputs,
                    name='name_of_model_2')

    model_3 = load_model('/kaggle/working/model3.h5')
    model_3 = Model(inputs=model_3.inputs,
                    outputs=model_3.outputs,
                    name='name_of_model_3')


    model_5 = load_model('/kaggle/working/model5.h5')
    model_5 = Model(inputs=model_5.inputs,
                    outputs=model_5.outputs,
                    name='name_of_model_5')

    model_7 = load_model('/kaggle/working/model7.h5')
    model_7 = Model(inputs=model_7.inputs,
                    outputs=model_7.outputs,
                    name='name_of_model_7')

#     model_9 = load_model('/kaggle/working/model9.h5')
#     model_9 = Model(inputs=model_9.inputs,
#                     outputs=model_9.outputs,
#                     name='name_of_model_9')



    models = [model_2, model_3, model_5,model_7]
    model_input = Input(shape=(224, 224, 3))
    model_outputs = [model(model_input) for model in models]
    ensemble_output = Average()(model_outputs)
    ensemble_model = Model(inputs=model_input, outputs=ensemble_output, name='ensemble')
    ensemble_model.compile(optimizer='adam',loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),metrics=['accuracy'])


In [ ]:
# ensemble_model.compile(optimizer='adam',loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),metrics=['accuracy'])


In [ ]:
history=ensemble_model.fit(
  train_ds,
  validation_data=test_ds,
  epochs=25)

In [ ]:
ensemble_evaluation = ensemble_model.evaluate(val_ds)
ensemble_evaluation[1]*100

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
train_label = np.concatenate([y for x, y in train_ds], axis=0)
print(train_label.shape)

test_label = np.concatenate([y for x, y in test_ds], axis=0)
print(test_label.shape)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
#Confution Matrix and Classification Report
import matplotlib.pyplot as plt
Y_pred = ensemble_model.predict_generator(test_ds, 1600)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
cm=confusion_matrix(test_label, y_pred)
print(cm)
print('Classification Report')
target_names = ['glioma', 'meningioma', 'notumor', 'pituitary']
print(classification_report(test_label, y_pred, target_names=target_names))

import seaborn as sns
sns.set(font_scale=1.0)
ax=sns.heatmap(cm, annot=True, cmap='summer', cbar=False, linewidths=3, linecolor='r', square=True, xticklabels=target_names,yticklabels=target_names,fmt='')
#sns.heatmap(cm, annot=True,annot_kws={"size": 22})
sns.set(font_scale = 2.0)
ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nActual Values')
ax.set_ylabel('Predicted Values ');